# Singularity

## Overview

- Package an analysis pipeline so that it runs on your laptop, in the cloud, and in a high performance computing (HPC) environment to produce the same result.
- Publish a paper and include a link to a container with all of the data and software that you used so that others can easily reproduce your results.
- Install and run an application that requires a complicated stack of dependencies with a few keystrokes.
- Create a pipeline or complex workflow where each individual program is meant to run on a different operating system.
- Singularity shines for scientific software running in an HPC environent.
- Singularity v3.0 was written in Go. 


### Docker

- Docker is built for running multiple containers on a single system and it allows containers to share common software features for efficiency. 
- Mature software with a large user community

Weaknesses
- Complex
- Not architected with security in mind
- Not built for HPC (but good for cloud)


## Instlling

Update packages
```:Ubuntu
sudo apt-get update

sudo apt-get install -y build-essential libssl-dev uuid-dev libgpgme11-dev \
   squashfs-tools libseccomp-dev wget pkg-config git cryptsetup debootstrap
```

Install GO
```
wget https://dl.google.com/go/go1.13.linux-amd64.tar.gz
sudo tar --directory=/usr/local -xzvf go1.13.linux-amd64.tar.gz
export PATH=/usr/local/go/bin:$PATH
```

Install Singularity
```
wget https://github.com/singularityware/singularity/releases/download/v3.5.3/singularity-3.5.3.tar.gz
tar -xzvf singularity-3.5.3.tar.gz
cd singularity
./mconfig
cd builddir
make
sudo make install
. etc/bash_completion.d/singularity
sudo cp etc/bash_completion.d/singularity /etc/bash_completion.d/
```

Basice command

In [7]:
!singularity --version

singularity version 3.5.3


In [1]:
# command summary

!singularity

Usage:
  singularity [global options...] <command>

Available Commands:
  build       Build a Singularity image
  cache       Manage the local cache
  capability  Manage Linux capabilities for users and groups
  config      Manage various singularity configuration (root user only)
  delete      Deletes requested image from the library
  exec        Run a command within a container
  inspect     Show metadata for an image
  instance    Manage containers running as services
  key         Manage OpenPGP keys
  oci         Manage OCI containers
  plugin      Manage Singularity plugins
  pull        Pull an image from a URI
  push        Upload image to the provided URI
  remote      Manage singularity remote endpoints
  run         Run the user-defined default command within a container
  run-help    Show the user-defined help for an image
  search      Search a Container Library for images
  shell       Run a shell within a container
  sif         siftool is a program for Singularity Imag

In [2]:
# singularity help <command>

!singularity help build

Build a Singularity image

Usage:
  singularity build [local options...] <IMAGE PATH> <BUILD SPEC>

Description:

  IMAGE PATH:

  When Singularity builds the container, output can be one of a few formats:

      default:    The compressed Singularity read only image format (default)
      sandbox:    This is a read-write container within a directory structure

  note: It is a common workflow to use the "sandbox" mode for development of the
  container, and then build it as a default Singularity image for production 
  use. The default format is immutable.

  BUILD SPEC:

  The build spec target is a definition (def) file, local image, or URI that can 
  be used to create a Singularity container. Several different local target 
  formats exist:

      def file  : This is a recipe for building a container (examples below)
      directory:  A directory structure containing a (ch)root file system
      image:      A local image on your machine (will convert to sif if
                  it 

In [6]:
# text run library

!singularity run library://godlovedc/funny/lolcow

FATAL:   Unable to handle library://godlovedc/funny/lolcow uri: unable to check if /home/jingwora/.singularity/cache/library/sha256.03187b702f874cf974f12d168cb741f05a3cfc6ae63ff7f111042b43864edca1/lolcow_latest.sif exists: hash does not match


In [ ]:
# download lolcow

!singularity pull library://godlovedc/funny/lolcow

In [1]:
# Singularity Image Format (SIF) file containing an image of a root level filesystem.

!ls lolcow_latest.sif

lolcow_latest.sif


In [4]:
# download the Docker version of lolcow
# Doing so may produce an error if the container already exists.

!singularity pull docker://godlovedc/lolcow

FATAL:   Image file already exists: "lolcow_latest.sif" - will not overwrite


In [5]:
!singularity shell lolcow_latest.sif

FATAL:   could not open image /home/jingwora/singularity/lolcow_latest.sif: SIF image /home/jingwora/singularity/lolcow_latest.sif is corrupted: wrong partition size


### Singularity flow
- create a writable container (called a sandbox)
- shell into the container with the --writable option and tinker with it interactively
- record changes that we like in our definition file
- rebuild the container from the definition file if we break it
- rinse and repeat until we are happy with the result
- rebuild the container from the final definition file as a read-only singularity image format (SIF) image for use in production

`build`
- To build a singularity container
- The build command installs an OS, sets up your container’s environment and installs the apps you need.
- To use the build command, we need a `definition file`, a set of instructions telling Singularity what software to install in the container.

In [1]:
# make directory
!mkdir lolcow

In [2]:
# copy sample file 
!cp singularity/examples/debian/Singularity lolcow/lolcow.def

Definition Files
- A Singularity Definition file is divided into two parts:

- Header: The core operating system to build within the container.

- Sections: Execute commands during the build process

- https://docs.sylabs.io/guides/3.5/user-guide/definition_files.html
- https://github.com/sylabs/examples

### Header
Bootstrap
- The only keyword that is required for every type of build is Bootstrap. 
- Preferred bootstrap agents
  - library (images hosted on the Container Library
  - docker (images hosted on Docker Hub)
  - shub (images hosted on Singularity Hub)
  - oras (images from supporting OCI registries)
  - scratch (a flexible option for building a container from scratch)

### Sections
- The main content of the bootstrap file is broken into sections.
- Different sections add different content or execute commands at different times during the build process. 
- Multiple sections of the same name can be included and will be appended to one another during the build process.
- Each section is defined by a `%` character followed by the name of the particular section.
- Sections that are executed at build time are executed with the `/bin/sh` interpreter and can accept `/bin/sh` options. 
- Sections that produce scripts to be executed at runtime can accept options intended for `/bin/sh`

In [3]:
# read file
!cat lolcow/lolcow.def

BootStrap: debootstrap
OSVersion: stable
MirrorURL: http://ftp.us.debian.org/debian/

%runscript
    echo "This is what happens when you run the container..."

%post
    echo "Hello from inside the container"
    apt-get -y --allow-unauthenticated install vim


Developing a new container
- Use this definition file to build our lolcow.img container.
```
sudo singularity build --sandbox lolcow lolcow.def
```

- This is telling Singularity to build a container called lolcow from the lolcow.def definition file.
- `sandbox` option in the command above tells Singularity that we want to build a special type of container (called a sandbox) for development purposes.
- `SIF` (singularity image format) fie format is defualt that uses squashfs for the file system. SIF files are compressed and immutable making them the best choice for reproducible, production-grade containers. 

- When your build finishes, you will have a basic Debian container saved in a local directory called `lolcow`.



Now let’s enter our new container and look around.
```
singularity shell lolcow
```
Your prompt change to Singularity>

Let’s try installing some software. If you enter a container without root privileges, you are unable to obtain root privileges within the container. 
```
Singularity> sudo apt-get update
bash: sudo: command not found
```

Let’s exit the container and re-enter as root.
```
exit
```

re-enter as root.
```
sudo singularity shell --writable lolcow
```
`--writable` option allows us to modify the container. 

Let’s try installing our software again.
```
Singularity> apt-get update
Singularity> apt-get install -y fortune cowsay lolcat
```
Now you should see the programs successfully installed. 

Let’s try running the demo in this new container.
```
Singularity> fortune | cowsay | lolcat
```
command not found! It looks like the programs were not added to our $PATH. 

Let add path and try again.
```
Singularity> export PATH=$PATH:/usr/games
Singularity> fortune | cowsay | lolcat

```

```
 __________________________________
/ Knock, knock!                    \
|                                  |
| Who's there? Sam and Janet.      |
|                                  |
| Sam and Janet who? Sam and Janet |
\ Evening...                       /
 ----------------------------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||
```
```
Singularity> exit
```

- We changed our path in this session, but those changes will disappear as soon as we exit the container.
- To make the changes permanent we should add them to the definition file and re-bootstrap the container.

Let’s update our definition file with the changes we made to this container.
- Add `apt-get update` to `%post`
- Create `%environment` with `export PATH=$PATH:/usr/games`

In [6]:
%%writefile lolcow/lolcow.def

BootStrap: debootstrap
OSVersion: stable
MirrorURL: http://ftp.us.debian.org/debian/

%runscript
    echo "This is what happens when you run the container..."

%post
    echo "Hello from inside the container"
    apt-get update
    apt-get -y install fortune cowsay lolcat

%environment
    export PATH=$PATH:/usr/games

Overwriting lolcow/lolcow.def


Let’s rebuild the container with the new definition file without `--sandbox` option.

```
$ sudo singularity build lolcow.sif lolcow.def
```
We will get `.sif` file.

In [8]:
# inspect command to read the runscript

!singularity inspect --deffile  lolcow/lolcow.sif


BootStrap: debootstrap
OSVersion: stable
MirrorURL: http://ftp.us.debian.org/debian/

%runscript
    echo "This is what happens when you run the container..."

%post
    echo "Hello from inside the container"
    apt-get update
    apt-get -y install fortune cowsay lolcat

%environment
    export PATH=$PATH:/usr/games



In practice, most people build containers from existing containers on the Container Library or on Docker Hub and use the `%post` section to modify those containers to suit their needs.
```
Bootstrap: docker
From: debian
```

You can also build a container from a base container on your local file system.
```
Bootstrap: localimage
From: /home/student/debian.sif
```

Alternatively, we can build without definition file.

```
$ singularity build debian1.sif library://debian
$ singularity build debian2.sif docker://debian
$ singularity build debian3.sif debian2.sif
```
Behind the scenes, Singularity creates a small definition file for each of these commands and then builds the corresponding container as you can see if you use the inspect `--deffile` command.

`--fakeroot` allowss you to pretend to be the root user inside of your container without actually granting singularity elevated privileges on host system. 

```
singularity build --fakeroot container.sif container.def
```

### Tags and hashes

Docker Hub and the Container Library both have a concept of a `tagged image` for developers to release several different versions.

```
$ singularity pull library://debian:9
```
definition file:
```
Bootstrap: library
From: debian:9
```
If you omit the `:<tag>`, you will get the container tagged with latest by default.

Some library has no default container which will fails.
```
$ singularity pull library://lolcow
FATAL:   While pulling library image: image lolcow:latest (amd64) does not exist in the library
```

Tags are not immutable and may change without warning. If you are interested in pulling the same container multiple times, you should pull by the `hash`.
```
$ singularity pull library://debian:sha256.b92c7fdfcc6152b983deb9fde5a2d1083183998c11fb3ff3b89c0efc7b240448
```

The syntax to do the same from Docker Hub is a bit different:
```
$ singularity pull docker://debian@sha256:f17410575376cc2ad0f6f172699ee825a51588f54f6d72bbfeef6e2fa9a57e2f
```

### trusted containers

General guidelines.

- Don’t build containers from untrusted sources or run them as root
- Review the runscript before you run it
- Use the `--no-home` and `--contain-all` options when running an unfamiliar container
- Establish your level of trust with a container

Docker Official Images
- https://docs.docker.com/docker-hub/official_images/

Docker Verified Publisher Program
- https://docs.docker.com/docker-hub/publish/

The Singularity Remote Builder offers ways to build your containers by composing a definition file or drag-and-drop using the web GUI.
- https://cloud.sylabs.io/builder

```
$ singularity remote login SylabsCloud
INFO:    Authenticating with remote: SylabsCloud
Generate an API Key at https://cloud.sylabs.io/auth/tokens, and paste here:
API Key:
INFO:    API Key Verified!
```

### Signing and sharing containers

You can generate a new PGP key with the key command like so:
```
$ singularity key newpair
Enter your name (e.g., John Doe) : Class Admin
Enter your email address (e.g., john.doe@example.com) : class.admin@mymail.com
Enter optional comment (e.g., development keys) : This is an example key for a class
Enter a passphrase :
Retype your passphrase :
Would you like to push it to the keystore? [Y,n] y
Generating Entity and OpenPGP Key Pair... done
Key successfully pushed to: https://keys.sylabs.io
```

This lets you cryptographically sign the container you just created with the sign command:
```
$ singularity sign alpine.sif
Signing image: alpine.sif
Enter key passphrase :
Signature created and applied to alpine.sif
```

The you can push it to the library like so:
```
$ singularity push alpine.sif library://godloved/base/alpine:latest
INFO:    Container is trusted - run 'singularity key list' to list your trusted keys
 2.59 MiB / 2.59 MiB [========================================================] 100.00% 10.72 MiB/s 0s
```

Then when others pull the container they can use the verify command to make sure that it has not been tampered with.
```
$ singularity verify alpine.sif
Container is signed by 1 key(s):

Verifying partition: FS:
73B905527AB1AA3929B6A736A47CBE85B37CB086
[LOCAL]   Class Admin (This is an example key for a class) <class.admin@mymail.com>
[OK]      Data integrity verified

INFO:    Container verified: alpine.sif
```

### Accessing Host Files with Bind Mounts

Within the container we can create and modify  files in our home directory (host system).

Example:
```
$ singularity shell lolcow.sif

Singularity> echo wutini > ~/jawa.txt

Singularity> cat ~/jawa.txt
wutini

Singularity> exit

$ cat ~/jawa.txt
wutini
```

There are several special directories that Singularity bind mounts into your container by default. These include:

- $HOME
- /tmp
- /proc
- /sys
- /dev

You can specify other directories to bind using the `--bind` option or the environmental variable `$SINGULARITY_BINDPATH`

to access a directory called /data from within our container.
```
$ sudo mkdir /data

$ sudo chown $USER:$USER /data

$ echo 'I am your father' > /data/vader.txt
```
using the --bind option to bind mount /data into the container.

```
$ singularity exec --bind /data lolcow.sif ls -l /data
total 4
-rw-rw-r-- 1 student student 17 Mar  2 00:51 vader.txt
```

bind mount multiple directories
```
$ singularity shell --bind src1:dest1,src2:dest2,src3:dest3 some.sif

$ export SINGULARITY_BINDPATH=src1:dest1,src2:dest2,src3:dest3
```

### Runscript

Consider an application that takes one file as input, analyzes the data in the file, and produces another file as output. 

Create input file
```
$ echo "The grass is always greener over the septic tank" > data/input

```
Analyze data
```
$ cat data/input | singularity exec lolcow.sif cowsay > data/output
```
Ouput
```
$ cat /data/output
 ______________________________________
/ The grass is always greener over the \
\ septic tank                          /
 --------------------------------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||
```

Use runscript
- Create a runscript inside the container

In [3]:
%%writefile lolcow/lolcow.def
BootStrap: library
From: debian:9

%runscript
    if [ $# -ne 2 ]; then
        echo "Please provide an input and an output file."
        exit 1
    fi
    cat $1 | cowsay > $2

%post
    apt-get update
    apt-get -y install fortune cowsay lolcat

%environment
    export PATH=$PATH:/usr/games

Overwriting lolcow/lolcow.def


- Rebuild out container to install the new runscript.
- `--force` option to force overwritten.
```
$ sudo singularity build --force lolcow.sif lolcow.def
```
to bind mount the /data directory into the container.
```
$ export SINGULARITY_BINDPATH=/data
$ ./lolcow.sif data/input data/output2
$ cat data/output2
 ______________________________________
/ The grass is always greener over the \
\ septic tank                          /
 --------------------------------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||
```

### Native Installation

Clear up lolcow
```
$ cd ~/lolcow

$ pwd # double check
/home/student/lolcow

$ sudo rm -rf * # caution! see NOTE above!
```

make a few new directories
```
$ mkdir libexec bin
```

create the contents of the libexec directory. It will contain the container, and a rather trickly little wrapper script.
```
$ singularity pull libexec/lolcow.sif library://godlovedc/funny/lolcow

$ cat >libexec/lolcow.sh<<"EOF"
#!/bin/bash
export SINGULARITY_BINDPATH="/data"
dir="$(dirname $(readlink -f ${BASH_SOURCE[0]}))"
img="lolcow.sif"
cmd=$(basename "$0")
arg="$@"
echo running: singularity exec "${dir}/${img}" $cmd $arg
singularity exec "${dir}/${img}" $cmd $arg
EOF

$ chmod 755 libexec/lolcow.sh
```

make a few symlinks in the bin directory.
```
$ ln -s ../libexec/lolcow.sh bin/fortune

$ ln -s ../libexec/lolcow.sh bin/cowsay

$ ln -s ../libexec/lolcow.sh bin/lolcat
```

Check files and folders

In [5]:
!tree lolcow

lolcow
├── bin
│   ├── cowsay -> ../libexec/lolcow.sh
│   ├── fortune -> ../libexec/lolcow.sh
│   └── lolcat -> ../libexec/lolcow.sh
└── libexec
    ├── lolcow.sh
    └── lolcow.sif

2 directories, 5 files


Now let’s see how it works:

```
$ cd ~

$ export PATH=$PATH:~/lolcow/bin

$ which cowsay fortune lolcat
/home/student/lolcow/bin/cowsay
/home/student/lolcow/bin/fortune
/home/student/lolcow/bin/lolcat

$ cowsay moo
running: singularity exec /home/student/lolcow/libexec/lolcow.sif cowsay moo
 _____
< moo >
 -----
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||

$ lolcat --help
running: singularity exec /home/student/lolcow/libexec/lolcow.sif lolcat --help

Usage: lolcat [OPTION]... [FILE]...

Concatenate FILE(s), or standard input, to standard output.
With no FILE, or when FILE is -, read standard input.

    --spread, -p <f>:   Rainbow spread (default: 3.0)
      --freq, -F <f>:   Rainbow frequency (default: 0.1)
      --seed, -S <i>:   Rainbow seed, 0 = random (default: 0)
       --animate, -a:   Enable psychedelics
  --duration, -d <i>:   Animation duration (default: 12)
     --speed, -s <f>:   Animation speed (default: 20.0)
         --force, -f:   Force color even when stdout is not a tty
       --version, -v:   Print version and exit
          --help, -h:   Show this message

Examples:
  lolcat f - g      Output f's contents, then stdin, then g's contents.
  lolcat            Copy standard input to standard output.
  fortune | lolcat  Display a rainbow cookie.

Report lolcat bugs to <http://www.github.org/busyloop/lolcat/issues>
lolcat home page: <http://www.github.org/busyloop/lolcat/>
Report lolcat translation bugs to <http://speaklolcat.com/>
Once you understand how it works, remove (or comment) the echo line in lolcow/libexec/lolcow.sh and even things like this should work without a hitch:

$ fortune | cowsay -n | lolcat
 ___________________________________________________________
< You never hesitate to tackle the most difficult problems. >
 -----------------------------------------------------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||
```

Run
```
$ fortune | cowsay -n | lolcat
 ___________________________________________________________
< You never hesitate to tackle the most difficult problems. >
 -----------------------------------------------------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||
```

### Miscellaneous Topics and FAQs

X11 and OpenGL
-  If you want to display X11 graphics you must install `xorg` within your container.

GPU computing
- In Singularity v2.3+ the experimental `--nv` option will look for NVIDIA libraries on the host system and automatically bind mount them to the container so that GPUs work seamlessly.

Using the network on the host system
- Network ports on the host system are accessible from within the container and work seamlessly. For example, you could install `ipython` within a container, start a jupyter notebook instance, and then connect to that instance using a browser running outside of the container on the host system or from another host.

A note on SUID programs and daemons
- Some programs need root privileges to run. These often include services or daemons that start via the `init.d` or `system.d` systems and run in the background. For instance, `sshd` the ssh daemon that listens on port 22 and allows another user to connect to your computer requires root privileges. You will not be able to run it in a container unless you start the container as root.

Long-running Instances
- Up to now all of our examples have run Singularity containers in the foreground. But what if you want to run a service like a `web server` or a `database` in a Singularity container in the background?

instance
- Use the `instance` command group to start and control container instances that run in the background. 

```
$ singularity instance.start lolcow.simg cow1
```
instance.list
- command to show the instances that are currently running
```
singularity instance.list
```

- We can connect to running instances using the `instance://` URI like so:
```
$ singularity shell instance://cow1
Singularity: Invoking an interactive shell within container...

Singularity lolcow.simg:~> ps -ef
UID        PID  PPID  C STIME TTY          TIME CMD
dave         1     0  0 19:05 ?        00:00:00 singularity-instance: dave [cow1]
dave         3     0  0 19:06 pts/0    00:00:00 /bin/bash --norc
dave         4     3  0 19:06 pts/0    00:00:00 ps -ef

Singularity lolcow.simg:~> exit
```

You can start multiple instances running in the background, as long as you give them unique names.
```
$ singularity instance.start lolcow.simg cow2

$ singularity instance.start lolcow.simg cow3

$ singularity instance.list
DAEMON NAME      PID      CONTAINER IMAGE
cow1             10794    /home/dave/lolcow.simg
cow2             10855    /home/dave/lolcow.simg
cow3             10885    /home/dave/lolcow.simg
```

You can stop individual instances using their unique names or stop all instances with the --all option.

```
$ singularity instance.stop cow1
Stopping cow1 instance of /home/dave/lolcow.simg (PID=10794)

$ singularity instance.stop --all
Stopping cow2 instance of /home/dave/lolcow.simg (PID=10855)
Stopping cow3 instance of /home/dave/lolcow.simg (PID=10885)
```

nginx

```
$ sudo singularity instance.start docker://nginx web
Docker image path: index.docker.io/library/nginx:latest
Cache folder set to /root/.singularity/docker
[3/3] |===================================| 100.0%
Creating container runtime...

$ sudo singularity instance.list
DAEMON NAME      PID      CONTAINER IMAGE
web              15379    /tmp/.singularity-runtime.MBzI4Hus/nginx
```


to start nginx running in the instance called web.
```
sudo singularity exec instance://web nginx
```

Now we have an nginx web server running on our localhost. We can verify that it is running with curl.
```
$ curl localhost
127.0.0.1 - - [02/Nov/2017:19:20:39 +0000] "GET / HTTP/1.1" 200 612 "-" "curl/7.52.1" "-"
<!DOCTYPE html>
<html>
<head>
<title>Welcome to nginx!</title>
<style>
    body {
        width: 35em;
        margin: 0 auto;
        font-family: Tahoma, Verdana, Arial, sans-serif;
    }
</style>
</head>
<body>
<h1>Welcome to nginx!</h1>
<p>If you see this page, the nginx web server is successfully installed and
working. Further configuration is required.</p>

<p>For online documentation and support please refer to
<a href="http://nginx.org/">nginx.org</a>.<br/>
Commercial support is available at
<a href="http://nginx.com/">nginx.com</a>.</p>

<p><em>Thank you for using nginx.</em></p>
</body>
</html>
```

stop all running instances
```
$ sudo singularity instance.stop --all
```

Ref:
- Doc: https://docs.sylabs.io/guides/3.0/user-guide/installation.html
- Tutorial: https://singularity-tutorial.github.io/

- Container registries
  - sylabs: https://cloud.sylabs.io/library
  - Docker: https://hub.docker.com/
  - Singularity Hub: http://datasets.datalad.org/?dir=/shub
  - Quay.io: https://quay.io/
  - NGC: https://ngc.nvidia.com/catalog
  - BioContainers: https://biocontainers.pro/#/registry